In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'jobdataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5335714%2F8865009%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T062804Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da113cda021407fb8a753d93d1529e62c5c756e607e411cf2c1fa14022abd9795fb211dcf373919ac7ac2d973eca05e9383d105ec28e71031f3a567f5a668631ce88f86f03d4a6b8958aa49092703fc00249c62957c66ff8fd00bdfcc41d6450983951e852c4c5595595b086a9d6a9663bfc88051cf90e0483d086608ec9ff9a365644155385ce90232cdc152a7c90484d99a095ceaa002f0888d7e0b0c4cd9696f2a63c75cd50547cb8066b0eb1961ac899e0c6c98fee635ed3972b0680e9f0d4e62aa691a7546cce849150ec3c951e5b41dd005369155118ed4617a2f1d68ce6a065657b79daa3ad20bf11ccef043f01463a1f7062a5c801242f517cd6780a0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 35213172 bytes downloaded
Downloaded and uncompressed: jobdataset
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobdataset/Combined_Jobs_Final.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
df_jobs = pd.read_csv("/kaggle/input/jobdataset/Combined_Jobs_Final.csv")

In [15]:
df_jobs.head()

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,...,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.00,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,...,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.00,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC
2,117,1,open,san-francisco-ca-machka-restaurants-corp-barte...,Bartender @ Machka Restaurants Corp.,Bartender,Machka Restaurants Corp.,San Francisco,California,CA,...,Food and Beverages,We are a popular Mediterranean wine bar and re...,NaN,11.00,NaN,NaN,Part-Time,NaN,2013-07-16 09:34:10 UTC,2014-08-16 15:35:37 UTC
3,121,1,open,brisbane-ca-teriyaki-house-server,Server @ Teriyaki House,Server,Teriyaki House,Brisbane,California,CA,...,Food and Beverages,● Serve food/drinks to customers in a profess...,NaN,10.55,NaN,NaN,Part-Time,NaN,2013-09-04 15:40:30 UTC,2014-08-16 15:35:38 UTC
4,127,1,open,los-angeles-ca-rosa-mexicano-sunset-kitchen-st...,Kitchen Staff/Chef @ Rosa Mexicano - Sunset,Kitchen Staff/Chef,Rosa Mexicano - Sunset,Los Angeles,California,CA,...,Food and Beverages,"Located at the heart of Hollywood, we are one ...",NaN,10.55,NaN,NaN,Part-Time,NaN,2013-07-17 15:26:18 UTC,2014-08-16 15:35:40 UTC


In [6]:
df_jobs['Job.Description']

0        Tacolicious' first Palo Alto store just opened...
1         \r\n\r\nNew French Brasserie in S.F. Financia...
2        We are a popular Mediterranean wine bar and re...
3         ● Serve food/drinks to customers in a profess...
4        Located at the heart of Hollywood, we are one ...
                               ...                        
84085    NJAHS stands for National Japanese American Hi...
84086    Weekend Brunch Line Cook \r\n● Other shifts ma...
84087    ONIGILLY (Japanese rice ball wraps) seeks outg...
84088    We are a popular Mediterranean restaurant in F...
84089     We are looking for a cashier! \r\n\r\n ● Take...
Name: Job.Description, Length: 84090, dtype: object

In [7]:
tfidf = TfidfVectorizer(stop_words="english")
df_jobs['Job.Description'] = df_jobs['Job.Description'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_jobs['Job.Description'])

In [8]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [16]:
indices = pd.Series(df_jobs.index, index = df_jobs['Title']).drop_duplicates()
indices

Title
Server @ Tacolicious                                               0
Kitchen Staff/Chef @ Claude Lane                                   1
Bartender @ Machka Restaurants Corp.                               2
Server @ Teriyaki House                                            3
Kitchen Staff/Chef @ Rosa Mexicano - Sunset                        4
                                                               ...  
Book Keeper @ National Japanese American Historical Society    84085
Kitchen Staff/Chef @ Emporio Rulli                             84086
Driver @ Onigilly                                              84087
Line Cook @ Machka Restaurants Corp.                           84088
Cashier @ Kazoo Restaurant                                     84089
Length: 84090, dtype: int64

In [10]:
indices['Server @ Teriyaki House']

3

In [11]:
def get_recommendation(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    sim_index = [i[0] for i in sim_scores]
    print(df_jobs["Title"].iloc[sim_index])
get_recommendation('Server @ Teriyaki House')

84046                                  Server @ Ramen Club
10781                                    Server @ Ca' Momi
10782                                  Server @ A La Turca
10783                      Server @ La Fontaine Restaurant
13390    Server @ Angelicas Fine Dining Bar And Enterta...
13391                    Server @ Colibri - Mexican Bistro
13458                                     Server @ COCO5OO
80806                     Server @ Thai Dishes on Broadway
83984                               Server @ Faz Sunnyvale
83989                                   Server @ L'Olivier
Name: Title, dtype: object


In [13]:
get_recommendation('Server @ Teriyaki House')

84046                                  Server @ Ramen Club
10781                                    Server @ Ca' Momi
10782                                  Server @ A La Turca
10783                      Server @ La Fontaine Restaurant
13390    Server @ Angelicas Fine Dining Bar And Enterta...
13391                    Server @ Colibri - Mexican Bistro
13458                                     Server @ COCO5OO
80806                     Server @ Thai Dishes on Broadway
83984                               Server @ Faz Sunnyvale
83989                                   Server @ L'Olivier
Name: Title, dtype: object


In [14]:
get_recommendation('Server @ COCO5OO')

10782                                  Server @ A La Turca
10783                      Server @ La Fontaine Restaurant
13390    Server @ Angelicas Fine Dining Bar And Enterta...
13391                    Server @ Colibri - Mexican Bistro
13458                                     Server @ COCO5OO
80806                     Server @ Thai Dishes on Broadway
83984                               Server @ Faz Sunnyvale
83989                                   Server @ L'Olivier
84007                                   Server @ 208 rodeo
84014                      Server @ Water Grill - Downtown
Name: Title, dtype: object
